In [7]:
from pyquery import PyQuery as pq
import requests
import re
import csv
from string import capwords
# import uucsv

In [8]:
votelist = []    # to be billno, chamber, membername, vote, points
floorvotevalue = 3

In [9]:
billlist = []
with open('billlist.csv', newline='') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
        if row['position'] == "bad":
            row['multiplier'] = -1
        elif row['position'] == "good":
            row['multiplier'] = 1
        else:
            print("Improper 'position' found for " + row['billno'] + ". What did you do?")
        billlist.append(row)

In [10]:
for row in billlist:
    for lowchamber in ("house", "senate"):
        if len(row[lowchamber]) > 0:   # If we have a valid URL
            billno = row['billno']
            chamber = capwords(lowchamber)
            print("Working on " + billno + " for the " + chamber)
            multiplier = row['multiplier']
            sourceurl = row[lowchamber]
            r = requests.get(sourceurl)
            html = r.content
            yeas = pq(html)("span#ctl00_ContentPlaceHolder1_lblYeas").text().strip()
            nays = pq(html)("span#ctl00_ContentPlaceHolder1_lblNays").text().strip()
            abstained = pq(html)("span#ctl00_ContentPlaceHolder1_lblAbstained").text().strip()
            missed = pq(html)("span#ctl00_ContentPlaceHolder1_lblMissed").text().strip()
            total = pq(html)("span#ctl00_ContentPlaceHolder1_lblTotal").text().strip()
            print(str(int(yeas)+int(nays)+int(abstained)+int(missed)) + " votes found. Y-N-A-M-Total:")    
            print(yeas, nays, abstained, missed, total)
            h = pq(html)("table#ctl00_ContentPlaceHolder1_dlVoteMember")
            for pol in pq(h)("td"):
                if len(pq(pol)("span")) > 0:
                    # print(pq(pol))
                    # print(pq(pol)("span")[0].text_content())
                    vote = pq(pol)("span")[0].text_content().strip()
                    # member = str(pq(pol)("span")[1].text_content().strip().encode("utf-8"))
                    member = pq(pol)("span")[1].text_content().strip()
                    if member == "NuÃ±ez":
                        print("HEY! PROBLEM WITH NUNEZ AGAIN")
                        member = u"Nuñez"
                        member = u"Nunez"
                    memberid = pq(pq(pol)("span")[1]).attr("id")
                    memberid = memberid.replace("ctl00_ContentPlaceHolder1_dlVoteMember_ctl", "")
                    memberid = memberid.replace("_lblVoterName", "")
                    points = 0
                    if vote == "Y":
                        points = floorvotevalue * multiplier
                    elif vote == "N":
                        points = floorvotevalue * multiplier * -1
                    else:
                        print("Vote \'" + vote + "\' found for " + member + " on " + billno)
                    entry = [billno, chamber, member, vote, str(points)]
                    # print(member, vote, memberid)
                    votelist.append(entry)
            if "Vote After Roll Call" in str(pq(html)):
                print("After-roll call votes need to be manually added for:\n\t"+ sourceurl)
    
    

Working on HB111 for the House
120 votes found. Y-N-A-M-Total:
113 3 0 4 116
Vote '-' found for Abruzzo on HB111
Vote '-' found for Santiago on HB111
Vote '-' found for Harrell on HB111
HEY! PROBLEM WITH NUNEZ AGAIN
Vote '-' found for Raulerson on HB111
After-roll call votes need to be manually added for:
	http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17008&BillId=56752&SessionId=83&
Working on HB111 for the Senate
40 votes found. Y-N-A-M-Total:
34 3 0 3 37
Vote '-' found for Hukill on HB111
Vote '-' found for Lee on HB111
Vote '-' found for Galvano on HB111
Working on SB118 for the House
120 votes found. Y-N-A-M-Total:
118 0 0 2 118
HEY! PROBLEM WITH NUNEZ AGAIN
Vote '-' found for Stafford on SB118
Vote '-' found for Plasencia on SB118
After-roll call votes need to be manually added for:
	http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17428&BillId=56694&SessionId=83&
Working on SB118 for the Senate
40 votes found. Y-N-A-M-Total:
34 0 0 6 34
Vot

In [11]:
# OK, so there can be fewer than 120 reps -- someone died, quit? Not marked absent. GOod to know.
# If number of voters is less than or equal to 40, it's Senate; otherwise, House

In [12]:
with open('autovotes.csv', 'w', newline='') as csvfile:
    put = csv.writer(csvfile)
    header = ["billno", "chamber", "member", "vote", "points"]
    put.writerow(header)
    for entry in votelist:
        put.writerow(entry)